- Title: Normalize JSON with Pandas
- Date: 2021-01-12 21:00:00 -0700
- Category: Tutorial
- Summary: An explanation about using the pandas json_normalize function to handle nested JSON data.

When processing nested `JSON` data into a flat structure for importing into a relational database, it can be tricky to structure the data into the right shape. [Pandas](https://pandas.pydata.org/) has a great tool for doing this called `pandas.json_normalize()` but the [documentation]((https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html?highlight=json_normalize)) doesn't make it obvious how to leverage its capabilities for handling nested data structures. 

I thought I could provide a brief example using some randomly generated survey response data (using the [Faker](https://faker.readthedocs.io/en/master/) library) to illustrate it's advantages.

## Setup

To start, I'm going to be using pandas and Faker so we'll import those. I'm also going to need to easily display the parsed json as well as the returned dataframes, so I'm importing the json module from the standard lib as well as some IPython notebook helpers for displaying dataframes as HTMl tables. 

In [67]:
import json
from IPython.display import display, HTML
from faker import Faker
from faker.providers import BaseProvider, date_time, internet
import pandas as pd

To make it easier to call the IPython display helper, we'll include this simple function as **syntactic sugar**. 

In [68]:
def print_df(df):
    display(HTML(df.to_html()))

## Fake Data
Faker doesn't have a built in provider for survey questions, so let's go ahead and add a simple one that creates non-sensical questions with a simple hack to the sentence provider.

In [69]:
class MyProvider(BaseProvider):
    def question(self):
        stems = ('Does', 'How does', 'Which', 'Why does')
        stem = Faker().random_choices(elements=stems, length=1)[0]
        sentence = Faker().sentence()
        sentence = sentence[0].lower() + sentence[1:]
        question = sentence.replace(".", "?")
        question = f"{stem} {question}"
        return question

Using this new question provider, we'll construct a few records of fake survey response data with some respondent level data like a respondent_id, survey_date, and respondent email. Within that we'll nest a list of responses which will in turn have it's own dictionary of data at the question level: id, question text, and choices. The choices list will be singular here, but assume it has that structure because the API this comes from has to also account for multi-select options and we'll need to parse it as a list regardless. *Depending on our analysis needs, this might also be a place where we'd want to keep these in a comma separated string, but for our purposes here we'll ignore that use case.*

In [79]:
fake = Faker()
fake.add_provider(MyProvider)
choices = ('Strongly Agree', 'Agree', 'Neutral', 'Disagree', 'Strongly Disagree')


sample_data = [
    {
        "respondent_id": fake.bothify(text="#?##??###?#"),
        "survey_date": fake.date(),
        "email": f"{fake.name()}@example.com",
        "responses": [
            {
                "question_id": fake.bothify(text="#??#??###?#"),
                "question_text": fake.question(),
                "choices": [
                    {
                        "choice": fake.random_choices(elements=choices, length=1)[0],
                        "number": fake.random_digit(),
                    }
                ]
            } for _ in range(5)
        ]
    } for _ in range(2) 
]

Let's print out a single record to see the resulting data structure that has been generated randomly.

In [80]:
print(json.dumps(sample_data[0], indent=2))

{
  "respondent_id": "6q96SI993a7",
  "survey_date": "2013-08-12",
  "email": "Tracy Foster@example.com",
  "responses": [
    {
      "question_id": "7Hq2ZB052H0",
      "question_text": "Why does several bad job hotel measure machine draw already?",
      "choices": [
        {
          "choice": "Strongly Disagree",
          "number": 4
        }
      ]
    },
    {
      "question_id": "7sF5Rc966u2",
      "question_text": "Why does prepare financial garden several power group?",
      "choices": [
        {
          "choice": "Disagree",
          "number": 6
        }
      ]
    },
    {
      "question_id": "3eW3tB947k7",
      "question_text": "How does scene high early still read?",
      "choices": [
        {
          "choice": "Agree",
          "number": 6
        }
      ]
    },
    {
      "question_id": "3xZ1up030p7",
      "question_text": "Does than not economy toward short than stand?",
      "choices": [
        {
          "choice": "Neutral",
          "num

## Reading with Pandas

As you can see below, simply reading this directly into a dataframe only parses the top level respondent data, but then keeps the responses data as a json array. Which isn't great for simple analysis. Could you load that as is in a jsonb field in PostgreSQL? Sure. If you like parsing json with SQL. Yuck!


In [81]:
df = pd.DataFrame(sample_data)
print_df(df)

,respondent_id,survey_date,email,responses
0,6q96SI993a7,2013-08-12,Tracy Foster@example.com,"[{'question_id': '7Hq2ZB052H0', 'question_text': 'Why does several bad job hotel measure machine draw already?', 'choices': [{'choice': 'Strongly Disagree', 'number': 4}]}, {'question_id': '7sF5Rc966u2', 'question_text': 'Why does prepare financial garden several power group?', 'choices': [{'choice': 'Disagree', 'number': 6}]}, {'question_id': '3eW3tB947k7', 'question_text': 'How does scene high early still read?', 'choices': [{'choice': 'Agree', 'number': 6}]}, {'question_id': '3xZ1up030p7', 'question_text': 'Does than not economy toward short than stand?', 'choices': [{'choice': 'Neutral', 'number': 1}]}, {'question_id': '4jt0Kn805O4', 'question_text': 'Which once yourself program?', 'choices': [{'choice': 'Neutral', 'number': 9}]}]"
1,3s64my293n2,2019-09-05,Mary Chen@example.com,"[{'question_id': '8ZY2IL469Z3', 'question_text': 'Does care training manage idea green produce machine?', 'choices': [{'choice': 'Disagree', 'number': 6}]}, {'question_id': '3jX8ix783b5', 'question_text': 'Which idea very care entire support rich?', 'choices': [{'choice': 'Neutral', 'number': 2}]}, {'question_id': '5ip4Sm635P6', 'question_text': 'Why does by collection leader well?', 'choices': [{'choice': 'Disagree', 'number': 6}]}, {'question_id': '4Hb8hr280u5', 'question_text': 'Which hospital father education build star through hospital?', 'choices': [{'choice': 'Agree', 'number': 5}]}, {'question_id': '3jJ6oU916M5', 'question_text': 'How does fire break material skill?', 'choices': [{'choice': 'Agree', 'number': 8}]}]"


## JSON Normalize
Thankfully there is the `json_normalize()` function, but it requires a little understanding to get it to satisfactorily parse flat. Simply passing it the sample data without any parameters results in a very familiar result that's get us no further than we started in the last attempt.

In [82]:
df3 = pd.json_normalize(sample_data)
print_df(df3)

,respondent_id,survey_date,email,responses
0,6q96SI993a7,2013-08-12,Tracy Foster@example.com,"[{'question_id': '7Hq2ZB052H0', 'question_text': 'Why does several bad job hotel measure machine draw already?', 'choices': [{'choice': 'Strongly Disagree', 'number': 4}]}, {'question_id': '7sF5Rc966u2', 'question_text': 'Why does prepare financial garden several power group?', 'choices': [{'choice': 'Disagree', 'number': 6}]}, {'question_id': '3eW3tB947k7', 'question_text': 'How does scene high early still read?', 'choices': [{'choice': 'Agree', 'number': 6}]}, {'question_id': '3xZ1up030p7', 'question_text': 'Does than not economy toward short than stand?', 'choices': [{'choice': 'Neutral', 'number': 1}]}, {'question_id': '4jt0Kn805O4', 'question_text': 'Which once yourself program?', 'choices': [{'choice': 'Neutral', 'number': 9}]}]"
1,3s64my293n2,2019-09-05,Mary Chen@example.com,"[{'question_id': '8ZY2IL469Z3', 'question_text': 'Does care training manage idea green produce machine?', 'choices': [{'choice': 'Disagree', 'number': 6}]}, {'question_id': '3jX8ix783b5', 'question_text': 'Which idea very care entire support rich?', 'choices': [{'choice': 'Neutral', 'number': 2}]}, {'question_id': '5ip4Sm635P6', 'question_text': 'Why does by collection leader well?', 'choices': [{'choice': 'Disagree', 'number': 6}]}, {'question_id': '4Hb8hr280u5', 'question_text': 'Which hospital father education build star through hospital?', 'choices': [{'choice': 'Agree', 'number': 5}]}, {'question_id': '3jJ6oU916M5', 'question_text': 'How does fire break material skill?', 'choices': [{'choice': 'Agree', 'number': 8}]}]"


A few optional parameters can be used here to parse the first nested array called `responses`. We can direct the pandas json parser to a specific key as the source of records. The `record_path` parameter takes either a string or list of strings to construct that path. The name of this parameter is a hint about how to think of this when passed as a list as we'll see later. 

We 

In [85]:
df4 = pd.json_normalize(
    sample_data, 
    record_path="responses", 
)
print_df(df4)

,question_id,question_text,choices
0,7Hq2ZB052H0,Why does several bad job hotel measure machine draw already?,"[{'choice': 'Strongly Disagree', 'number': 4}]"
1,7sF5Rc966u2,Why does prepare financial garden several power group?,"[{'choice': 'Disagree', 'number': 6}]"
2,3eW3tB947k7,How does scene high early still read?,"[{'choice': 'Agree', 'number': 6}]"
3,3xZ1up030p7,Does than not economy toward short than stand?,"[{'choice': 'Neutral', 'number': 1}]"
4,4jt0Kn805O4,Which once yourself program?,"[{'choice': 'Neutral', 'number': 9}]"
5,8ZY2IL469Z3,Does care training manage idea green produce machine?,"[{'choice': 'Disagree', 'number': 6}]"
6,3jX8ix783b5,Which idea very care entire support rich?,"[{'choice': 'Neutral', 'number': 2}]"
7,5ip4Sm635P6,Why does by collection leader well?,"[{'choice': 'Disagree', 'number': 6}]"
8,4Hb8hr280u5,Which hospital father education build star through hospital?,"[{'choice': 'Agree', 'number': 5}]"
9,3jJ6oU916M5,How does fire break material skill?,"[{'choice': 'Agree', 'number': 8}]"


But when we direct the parser to just unpack the `reponses` array, we lose our data from the level above. Pandas can be instructed to keep this by giving it a list of metadata to repeat for each record it unpacks from the level above. We use the `meta` parameter and pass it a list of the fields to include.

In [87]:
df4 = pd.json_normalize(
    sample_data, 
    record_path="responses", 
     meta=[
        "respondent_id", 
        "survey_date",
        "email", 
    ],
)
print_df(df4)
   

,question_id,question_text,choices,respondent_id,survey_date,email
0,7Hq2ZB052H0,Why does several bad job hotel measure machine draw already?,"[{'choice': 'Strongly Disagree', 'number': 4}]",6q96SI993a7,2013-08-12,Tracy Foster@example.com
1,7sF5Rc966u2,Why does prepare financial garden several power group?,"[{'choice': 'Disagree', 'number': 6}]",6q96SI993a7,2013-08-12,Tracy Foster@example.com
2,3eW3tB947k7,How does scene high early still read?,"[{'choice': 'Agree', 'number': 6}]",6q96SI993a7,2013-08-12,Tracy Foster@example.com
3,3xZ1up030p7,Does than not economy toward short than stand?,"[{'choice': 'Neutral', 'number': 1}]",6q96SI993a7,2013-08-12,Tracy Foster@example.com
4,4jt0Kn805O4,Which once yourself program?,"[{'choice': 'Neutral', 'number': 9}]",6q96SI993a7,2013-08-12,Tracy Foster@example.com
5,8ZY2IL469Z3,Does care training manage idea green produce machine?,"[{'choice': 'Disagree', 'number': 6}]",3s64my293n2,2019-09-05,Mary Chen@example.com
6,3jX8ix783b5,Which idea very care entire support rich?,"[{'choice': 'Neutral', 'number': 2}]",3s64my293n2,2019-09-05,Mary Chen@example.com
7,5ip4Sm635P6,Why does by collection leader well?,"[{'choice': 'Disagree', 'number': 6}]",3s64my293n2,2019-09-05,Mary Chen@example.com
8,4Hb8hr280u5,Which hospital father education build star through hospital?,"[{'choice': 'Agree', 'number': 5}]",3s64my293n2,2019-09-05,Mary Chen@example.com
9,3jJ6oU916M5,How does fire break material skill?,"[{'choice': 'Agree', 'number': 8}]",3s64my293n2,2019-09-05,Mary Chen@example.com


## We must go deeper!

That works for the most part, but we still have that annoying **choices** json array that would be nice to split out into columns. 

```python
df5 = pd.json_normalize(
    sample_data, 
    record_path="choices", 
    meta=[
        "respondent_id", 
        "survey_date",
        "email", 
    ],
)
```

Simply passing the choices field to the `record_path` param results in a `KeyError` though. This is because the **choices** field is actually nested in the **responses** field. So pandas need us to construct a path to reach it. We can get to it by passing each key as a record in the list to construct a path. Here that looks like `["responses", "choices"]`.

In [91]:
df5 = pd.json_normalize(
    sample_data, 
    record_path=["responses", "choices"], 
    meta=[
        "respondent_id", 
        "survey_date",
        "email", 
    ],
)
print_df(df5)

,choice,number,respondent_id,survey_date,email
0,Strongly Disagree,4,6q96SI993a7,2013-08-12,Tracy Foster@example.com
1,Disagree,6,6q96SI993a7,2013-08-12,Tracy Foster@example.com
2,Agree,6,6q96SI993a7,2013-08-12,Tracy Foster@example.com
3,Neutral,1,6q96SI993a7,2013-08-12,Tracy Foster@example.com
4,Neutral,9,6q96SI993a7,2013-08-12,Tracy Foster@example.com
5,Disagree,6,3s64my293n2,2019-09-05,Mary Chen@example.com
6,Neutral,2,3s64my293n2,2019-09-05,Mary Chen@example.com
7,Disagree,6,3s64my293n2,2019-09-05,Mary Chen@example.com
8,Agree,5,3s64my293n2,2019-09-05,Mary Chen@example.com
9,Agree,8,3s64my293n2,2019-09-05,Mary Chen@example.com


But when we do that, we lose our **question_id** and **question_text** fields. That's because we need to add them in the meta list and pass their paths like the record path param. See below.

In [92]:
df5 = pd.json_normalize(
    sample_data, 
    record_path=["responses", "choices"], 
    meta=[
        "respondent_id", 
        "survey_date",
        "email", 
        ["responses", "question_id"],
        ["responses", "question_text"],
    ],
)
print_df(df5)

,choice,number,respondent_id,survey_date,email,responses.question_id,responses.question_text
0,Strongly Disagree,4,6q96SI993a7,2013-08-12,Tracy Foster@example.com,7Hq2ZB052H0,Why does several bad job hotel measure machine draw already?
1,Disagree,6,6q96SI993a7,2013-08-12,Tracy Foster@example.com,7sF5Rc966u2,Why does prepare financial garden several power group?
2,Agree,6,6q96SI993a7,2013-08-12,Tracy Foster@example.com,3eW3tB947k7,How does scene high early still read?
3,Neutral,1,6q96SI993a7,2013-08-12,Tracy Foster@example.com,3xZ1up030p7,Does than not economy toward short than stand?
4,Neutral,9,6q96SI993a7,2013-08-12,Tracy Foster@example.com,4jt0Kn805O4,Which once yourself program?
5,Disagree,6,3s64my293n2,2019-09-05,Mary Chen@example.com,8ZY2IL469Z3,Does care training manage idea green produce machine?
6,Neutral,2,3s64my293n2,2019-09-05,Mary Chen@example.com,3jX8ix783b5,Which idea very care entire support rich?
7,Disagree,6,3s64my293n2,2019-09-05,Mary Chen@example.com,5ip4Sm635P6,Why does by collection leader well?
8,Agree,5,3s64my293n2,2019-09-05,Mary Chen@example.com,4Hb8hr280u5,Which hospital father education build star through hospital?
9,Agree,8,3s64my293n2,2019-09-05,Mary Chen@example.com,3jJ6oU916M5,How does fire break material skill?


One last tweak: some databases (like MS SQL) don't like naming columns with that period in the name. As a work around you can give the `json_normalize` function a custom separator such as an underscore instead. 

In [95]:
df5 = pd.json_normalize(
    sample_data, 
    sep="_",
    record_path=["responses", "choices"], 
    meta=[
        "respondent_id", 
        "survey_date",
        "email", 
        ["responses", "question_id"],
        ["responses", "question_text"]
    ],
)
print_df(df5)

,choice,number,respondent_id,survey_date,email,responses_question_id,responses_question_text
0,Strongly Disagree,4,6q96SI993a7,2013-08-12,Tracy Foster@example.com,7Hq2ZB052H0,Why does several bad job hotel measure machine draw already?
1,Disagree,6,6q96SI993a7,2013-08-12,Tracy Foster@example.com,7sF5Rc966u2,Why does prepare financial garden several power group?
2,Agree,6,6q96SI993a7,2013-08-12,Tracy Foster@example.com,3eW3tB947k7,How does scene high early still read?
3,Neutral,1,6q96SI993a7,2013-08-12,Tracy Foster@example.com,3xZ1up030p7,Does than not economy toward short than stand?
4,Neutral,9,6q96SI993a7,2013-08-12,Tracy Foster@example.com,4jt0Kn805O4,Which once yourself program?
5,Disagree,6,3s64my293n2,2019-09-05,Mary Chen@example.com,8ZY2IL469Z3,Does care training manage idea green produce machine?
6,Neutral,2,3s64my293n2,2019-09-05,Mary Chen@example.com,3jX8ix783b5,Which idea very care entire support rich?
7,Disagree,6,3s64my293n2,2019-09-05,Mary Chen@example.com,5ip4Sm635P6,Why does by collection leader well?
8,Agree,5,3s64my293n2,2019-09-05,Mary Chen@example.com,4Hb8hr280u5,Which hospital father education build star through hospital?
9,Agree,8,3s64my293n2,2019-09-05,Mary Chen@example.com,3jJ6oU916M5,How does fire break material skill?


Lastly, we'll record the columns to start from the least nested level on the left all the way to the most nested on the right for easier interpretation. 

In [76]:
column_order = ["respondent_id", "survey_date", "email", "responses_question_id", "responses_question_text", "choice", "number"]
df6 = df5[column_order]
print_df(df6)

,respondent_id,survey_date,email,responses_question_id,responses_question_text,choice,number
0,8l80gS238W2,1982-10-31,Dan Thomas@example.com,9ZR5ZR439f4,How does in something of different?,Disagree,0
1,8l80gS238W2,1982-10-31,Dan Thomas@example.com,9DK0Am643P0,How does impact same move value water?,Strongly Disagree,8
2,8l80gS238W2,1982-10-31,Dan Thomas@example.com,0NE2jJ853s2,Why does including exactly thus player?,Strongly Agree,7
3,8l80gS238W2,1982-10-31,Dan Thomas@example.com,1mK5tX361J2,Why does training deep day group let president?,Agree,1
4,8l80gS238W2,1982-10-31,Dan Thomas@example.com,3PM4xL997q4,Does her ok lot hour?,Agree,1
5,8o18qO450A6,2015-07-19,Katrina Gross@example.com,9Of3Hc997V6,Why does student mouth any they hair positive?,Neutral,6
6,8o18qO450A6,2015-07-19,Katrina Gross@example.com,7Ql0Qg952F3,Does finally majority miss?,Disagree,9
7,8o18qO450A6,2015-07-19,Katrina Gross@example.com,5GL0iD950d2,Does machine whatever floor mission budget court production?,Strongly Disagree,5
8,8o18qO450A6,2015-07-19,Katrina Gross@example.com,5Ie6OO091n8,Why does nearly nothing owner per?,Disagree,3
9,8o18qO450A6,2015-07-19,Katrina Gross@example.com,9Ca8PI956p7,Does ready whatever quite plant forward yes summer?,Neutral,8


## Parsed and read to import

At this point our data is in a tabular format and ready to import into a database table with something like pandas [to_sql()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html?highlight=to_sql) function, but we'll save that for another post. 